# 一、文档嵌入模型【Text Embedding Model】

## 1、嵌入模型概述

- Text Embedding Models:文档嵌入模型,提供将文本编码为向量的能力,即文档向量化
     文档写入和用户查询匹配前都会先执行文档嵌入编码,即向量化
- LangChain从开源到专有API,提供了超过25种不同的嵌入提供商和方法的集成，
- Hugging Face等开源社区提供了一些文本向量化模型(例如BGE),效果比闭源且调用API的向量化模型效果好，并且向量化模型参数量小，在CPU上即可运行。
- 所以，这里推荐在开发RAG应用的过程中，使用开源的文本向量化模型。此外,开源模型还可以根据应用场景下收集的数据对模型进行微调.提高模型效果。
- LangChain中针对向量化模型的封装提供了两种接口,一种针对文档的向量化(embed_documents),一种针对 句子的向量化embed_query 。

## 2、句子/文档嵌入

In [4]:
from langchain_openai import OpenAIEmbeddings
import os, dotenv

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 嵌入句子【embed_query函数】
text = "Text Embedding Models:文档嵌入模型，提供将文本编码为向量的能力,即文档向量化,文档写入和 用户查询匹配 前都会先执行文档嵌入编码,即向量化"
text_vector = embedding_model.embed_query(text)

print(len(text_vector))  # 输出嵌入向量的维度
print(f"text_vector: {text_vector[:100]}")

# 待嵌入的文本列表【embed_documents函数】, 接收的参数是字符串数组。
texts = [
    "Hi there!",
    "Oh, hello!",
    "What hobbies do you have? ",
    "I like swimming, programming, and writing!"
]
# 生成嵌入向量
docs_vector = embedding_model.embed_documents(texts)

for i in range(len(docs_vector)):
    print(f"{texts[i]} => {docs_vector[i][:5]}",end="\n\n")

1536
text_vector: [-0.029042674228549004, 0.06226688623428345, -0.005592805799096823, -0.025355318561196327, 0.007365207187831402, -0.00924689881503582, 0.01829422451555729, 0.018246706575155258, -0.003281080862507224, -0.023796746507287025, 0.034155555069446564, 0.03459271416068077, -0.021762998774647713, -0.0022891538683325052, 0.004088877700269222, 0.016583595424890518, 0.03512490913271904, -0.0029532103799283504, 0.0008398711797781289, 0.013342903926968575, 0.001646480173803866, 0.025317305698990822, -0.014245736412703991, 0.002846296178176999, 0.04447634890675545, -0.017353378236293793, -0.005169900134205818, 0.05987200513482094, 0.03767184540629387, -0.08401087671518326, -0.04006672650575638, -0.0108339823782444, 0.008196762762963772, -0.018911950290203094, 0.009845619089901447, 0.022732354700565338, -0.010643912479281425, 0.031038407236337662, -0.0025611913297325373, -0.0027797718066722155, -0.00574486143887043, 0.05090070888400078, -0.01094802375882864, 0.03812801465392113, 0.0

- 加载csv文件并拆分成文本块:

In [7]:
from langchain_community.document_loaders import CSVLoader

# 情况1:
loader = CSVLoader("./document/load/03-load.csv", encoding="utf-8")
# 加载并切分文档
docs = loader.load_and_split()
#print(len(docs))
# 存放的是每一个chrunk的embedding。
embeded_docs = embedding_model.embed_documents([doc.page_content for doc in docs])
print(len(embeded_docs))
# 表示的是每一个chrunk的embedding的维度
print(len(embeded_docs[0]))

for i in range(len(embeded_docs)):
    print(f"{docs[i]} => {embeded_docs[i][:10]}",end="\n\n")


4
1536
page_content='id: 1
title: Introduction to Python
content: Python is a popular programming language.
author: Guido van Rossum' metadata={'source': './document/load/03-load.csv', 'row': 0} => [0.02165742963552475, -0.044537920504808426, 0.022788185626268387, -0.002097090007737279, -0.007367218378931284, -0.054230112582445145, -0.029330413788557053, 0.05589162930846214, -0.014088290743529797, 0.04375331476330757]

page_content='id: 2
title: Data Science Basics
content: Data science involves statistics and machine learning.
author: Jane Smith' metadata={'source': './document/load/03-load.csv', 'row': 1} => [0.01213858276605606, 0.0008625349146313965, 0.0406014658510685, -0.03944423794746399, 0.01269257441163063, 0.006567884236574173, -0.017186066135764122, 0.02560674585402012, -0.010735135525465012, 0.03511079028248787]

page_content='id: 3
title: Web Development
content: HTML, CSS and JavaScript are core web technologies.
author: Mike Johnson' metadata={'source': './document/load/

# 二、向量 & 向量数据库

## 1、向量 & 向量数据库的概念理解

- 向量: 将文本、图像、音频等非结构化数据（比如一篇文档、一张图片）, 通过 Embedding 嵌入模型（如 BERT、Sentence-BERT、CLIP）转换成的高维数值数组（比如 768 维、1536 维），这个数组就叫 “向量”,向量本质是对数据 “语义 / 特征” 的数学表达。

- 向量数据库: 专门用于存储、索引、查询向量数据的数据库，核心能力是 “给定一个查询向量，快速找出数据库中最相似的 Top N 个向量”（即 “近似最近邻查询”，ANN），并返回对应的原始数据（如文档片段、图片）
    - 传统数据库解决 “找精确匹配的数据”(比如 “查用户 ID=123 的订单”), 适用于用户信息、订单管理等场景
    - 向量数据库解决 “找意思相近的数据”(比如 “查与‘如何优化 ES 慢查询’语义相似的文档”),适用于RAG 知识库检索、推荐系统、图像检索等场景

## 2、为什么使用向量数据库

- 在做 RAG、Agent 等 AI 应用时，直接用传统数据库或内存检索会遇到 3 个致命问题，向量数据库刚好对症下药:
    - 1）非结构化数据无法 “语义匹配”: 比如你想让 AI 回答 “如何搭建 RAG 系统”，传统数据库只能通过 “关键词匹配”（如 “RAG”“搭建”）找文档，容易漏掉 “基于 LangChain 构建知识库检索系统” 这类语义相近但关键词不同的内容；而向量数据库会把问题转换成向量，直接召回语义最相似的文档，理解更精准。
    - 2）高维向量查询太慢: 如果把 100 万条文档转换成 768 维向量存在数组里，暴力遍历计算相似度需要几秒甚至几分钟，完全无法满足 AI 应用的实时响应需求（比如用户提问后 1 秒内返回答案;向量数据库通过 “索引算法”（如 HNSW、IVF），能把查询时间压缩到毫秒级。
    - 3）数据规模扩容困难: 当向量数据达到千万、亿级时，内存放不下，传统数据库缺乏高效的分布式存储和检索方案;主流向量数据库支持水平扩容，能应对大规模数据场景（比如企业级知识库）。

## 3、向量数据库的核心特性

- 1）高维向量支持:能轻松存储几百到几千维的向量（比如 OpenAI 的 text-embedding-3-large 输出 3072 维向量,Milvus、Chroma 都能兼容）
- 2）高效索引算法：核心是 “用空间换时间”，提前对向量建立索引，加速查询:
    - 常用索引: HNSW（平衡速度和准确率，最常用）、IVF（适合大规模数据，需调参）、FAISS（Facebook 开源，适合小规模内存场景）
- 3）分布式能力: 支持分片存储、副本备份，解决单节点存储和性能瓶颈（比如 Milvus 支持跨节点负载均衡，适合亿级向量场景）
- 4）元数据过滤: 可给向量附加结构化元数据（如 “文档类型 = 技术文档”“创建时间 = 2024-01-01”），查询时先过滤元数据再做向量匹配（比如 “只找 2024 年后的 RAG 相关文档”），进一步提升效率
- 5）多模态支持:不仅能处理文本向量，还能存储图像、音频向量（比如用 CLIP 模型转换图片向量，实现 “以图搜图”）。

## 4、向量数据库技术选型

- LangChain提供了超过 50种 不同向量存储（Vector Stores）的集成，从开源的本地向量存储 到 云托管的私有向量存储，允许你选择最适合需求的向量存储。
- LangChain支持的向量存储可参考 VectorStore 接口和实现

- 主流向量数据库选型对比（开发者实操推荐）
| 数据库名称   | 开源 / 闭源    | 部署方式              | 优势                                     | 适用场景                       |
|------------|--------------|---------------------|----------------------------------------|------------------------------|
| Milvus     | 开源         | 本地 / 云 / 容器        | 分布式能力强，支持亿级向量，功能全面（元数据过滤、多模态）  | 企业级 RAG、大规模知识库            |
| Chroma     | 开源         | 本地 / 轻量部署         | 上手简单，API 友好，适合快速开发                   | 个人项目、中小型 RAG（如博客助手）     |
| Pinecone   | 闭源（云服务） | 云端托管               | 零运维，scalability 强                   | 不想搭环境的快速验证场景             |
| FAISS      | 开源（库）     | 本地嵌入应用             | 轻量，查询速度快（小规模）                        | 嵌入式场景（如客户端 AI 工具）        |
| Qdrant     | 开源         | 本地 / 云               | 支持地理空间索引，多模态友好                       | 结合地理位置的推荐、多模态检索         |

- 选型建议:
    - 快速验证原型(比如个人做抖音AI文案助手): 选 Chroma（一行命令启动，Python/JS SDK 都有）。
    - 企业级项目(比如公司内部技术知识库):选 Milvus（分布式、高可用，支持横向扩展）。
    - 不想管服务器: 选 Pinecone（云端托管，按用量付费）

## 5、向量存储

- 文档加载、嵌入、分块、相似性检索的初步实践

In [27]:
# 举例：将分割后的文本，使用 OpenAI 嵌入模型获取嵌入向量，并存储在 Chroma 中
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# 1.获取嵌入模型
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 2.创建文档加载器,并加载指定的文档
loader = TextLoader(file_path="document/load/09-embedding-doc.txt", encoding="utf-8")
# 3.获取文档拆分器(并指定每个分块的大小和重叠大小)
splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

# texts = loader.load()
# # 4.切分文档
# docs = splitter.split_documents(texts)

# 加载并拆分文档
docs = loader.load_and_split(splitter)

# 5.将切分后的文本块 经由嵌入模型生成向量后,向量和原文本块一起存储到Chroma向量数据库
vector_db = Chroma.from_documents(docs, embedding_model)

# 6.查询向量数据库,进行相似度检索
query = "请简单介绍下人工智能的关键技术"
result_docs = vector_db.similarity_search(query,2)   # 返回最相关的2个文档

print(f"len: {len(result_docs)}")

for i in range(len(result_docs)):
    print(f"page_content {i}: {result_docs[i].page_content}", end="\n\n")

len: 2
page_content 0: 3. 人工智能的核心技术
3.1 机器学习
机器学习是人工智能的核心技术之一，通过算法使计算机从数据中学习并做出决策。常见的机器学习算法包括监督学习、无监督学习和强化学习。监督学习通过标记数据进行训练，无监督学习则从未标记数据中寻找模式，强化学习则通过与环境交互来优化决策。
3.2 深度学习

page_content 1: 人工智能的核心技术
1) 机器学习
机器学习是人工智能的核心技术之一，通过算法使计算机从数据中学习并做出决策。常见的机器学习算法包括监督学习、无监督学习和强化学习。监督学习通过标记数据进行训练，无监督学习则从未标记数据中寻找模式，强化学习则通过与环境交互来优化决策。
2) 深度学习
深度学习是机器学习的一个子领域，通过多层神经网络进行特征提取和模式识别。深度学习在图像识别、自然语言处理、语音识别等领域取得了显著成果。常见的深度学习模型包括卷积神经网络（CNN）、循环神经网络（RNN）和长短期记忆网络（LSTM）。
3) 自然语言处理
自然语言处理（NLP）是人工智能的一个重要分支，致力于使计算机能够理解和生成人类语言。NLP技术广泛应用于机器翻译、情感分析、文本分类等领域。近年来，基于深度学习的NLP模型（如BERT、GPT）在语言理解任务中取得了突破性进展。
4) 计算机视觉
计算机视觉是人工智能的另一个重要分支，致力于使计算机能够理解和处理图像和视频。计算机视觉技术广泛应用于图像识别、目标检测、人脸识别等领域。深度学习模型（如CNN）在计算机视觉任务中取得了显著成果。



- Chroma主要有两种存储模式: 内存模式 和 持久化模式。
- 当使用persist_directory参数时，数据会保存到指定目录;如果没有指定，则默认使用内存存储

In [28]:
Chroma.from_documents(docs, embedding_model, persist_directory="./chroma-embeddings")

## 6、向量检索

### 1）相似性检索(similarity_search)

In [31]:
from langchain_core.documents import Document

# 2.定义文档
raw_documents = [
    Document(
        page_content="葡萄是一种常见的水果，属于葡萄科葡萄属植物。它的果实呈圆形或椭圆形，颜色有绿色、紫色、红色等多种。葡萄富含维生素C和抗氧化物质，可以直接食用或酿造成葡萄。",
        metadata={"source": "水果", "type": "植物"}),
    Document(
        page_content="白菜是十字花科蔬菜，原产于中国北方。它的叶片层层包裹形成紧密的球状，口感清脆微甜。白菜富含膳食纤维和维生素K，常用于制作泡菜、炒菜或煮汤。",
        metadata={"source": "蔬菜", "type": "植物"}),
    Document(
        page_content="狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。",
        metadata={"source": "动物", "type": "哺乳动物"}),
    Document(
        page_content="猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。",
        metadata={"source": "动物", "type": "哺乳动物"}),
    Document(
        page_content="人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。",
        metadata={"source": "生物", "type": "灵长类"}),
    Document(
        page_content="太阳是太阳系的中心恒星，直径约139万公里，主要由氢和氦组成。它通过核聚变反应产生能量，为地球提供光和热。太阳活动周期约为11年，会影响地球气候。",
        metadata={"source": "天文", "type": "恒星"}),
    Document(
        page_content="长城是中国古代的军事防御工程，总长度超过2万公里。它始建于春秋战国时期，秦朝连接各段，明朝大规模重修。长城是世界文化遗产和人类建筑奇迹。",
        metadata={"source": "历史", "type": "建筑"}),
    Document(
        page_content="量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。",
        metadata={"source": "物理", "type": "科学"}),
    Document(
        page_content="《红楼梦》是中国古典文学四大名著之一，作者曹雪芹。小说以贾、史、王、薛四大家族的兴衰为背景，描绘了贾宝玉与林黛玉的爱情悲剧，反映了封建社会的种种矛盾。",
        metadata={"source": "文学", "type": "小说"}),
    Document(
        page_content="新冠病毒（SARS-CoV-2）是一种可引起呼吸道疾病的冠状病毒。它通过飞沫传播，主要症状包括发热、咳嗽、乏力。疫苗和戴口罩是有效的预防措施。",
        metadata={"source": "医学", "type": "病毒"})
]
# 3. 创建嵌入模型
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
# 4.创建向量数据库
vector_db = Chroma.from_documents(
    documents=raw_documents,
    embedding=embedding_model,
    persist_directory="./chroma-embeddings",
)

# 5. 检索示例（返回前3个最相关结果）
query = "哺乳动物"
docs = vector_db.similarity_search(query, k=3) # k=3表示返回3个最相关文档
print(f"查询: '{query}' 的结果:")
for i, doc in enumerate(docs, 1):
    print(f"\n结果 {i}:")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")

查询: '哺乳动物' 的结果:

结果 1:
内容: 猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。
元数据: {'type': '哺乳动物', 'source': '动物'}

结果 2:
内容: 狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。
元数据: {'type': '哺乳动物', 'source': '动物'}

结果 3:
内容: 人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。
元数据: {'type': '灵长类', 'source': '生物'}


### 2）对问题向量查询(similarity_search_by_vector)

In [33]:
# 嵌入模型生成查询向量
query_vector = embedding_model.embed_query(text="哺乳动物")
# 嵌入的查询向量作为参数传入,实现对问题向量的查询
docs = vector_db.similarity_search_by_vector(query_vector, k=3)

print(f"查询: '{query}' 的结果:")
for i,doc in enumerate(docs, 1):
    print(f"\n结果 {i}:")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")


查询: '哺乳动物' 的结果:

结果 1:
内容: 猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。
元数据: {'source': '动物', 'type': '哺乳动物'}

结果 2:
内容: 狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。
元数据: {'source': '动物', 'type': '哺乳动物'}

结果 3:
内容: 人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。
元数据: {'type': '灵长类', 'source': '生物'}


### 3）相似性 & 元数据过滤检索(filter)

In [37]:
query = "哺乳动物"
docs = vector_db.similarity_search(
    query=query,
    k=3,
    filter={"source":"动物"}
)
print(f"查询: '{query}' 的结果:")
for i, doc in enumerate(docs, 1):
    print(f"\n结果 {i}:")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")


query = "微观粒子运动规律"
docs = vector_db.similarity_search(
    query=query,k=2,
    filter={"type":"科学"}
)
print("=" * 20)
print(f"查询: '{query}' 的结果:")
for i, doc in enumerate(docs, 1):
    print(f"\n结果 {i}:")
    print(f"内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")

查询: '哺乳动物' 的结果:

结果 1:
内容: 猫是小型肉食性哺乳动物，性格独立但也能与人类建立亲密关系。它们夜视能力极强，擅长捕猎老鼠。家猫的品种包括波斯猫、暹罗猫等，毛色和花纹多样。
元数据: {'type': '哺乳动物', 'source': '动物'}

结果 2:
内容: 狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。
元数据: {'type': '哺乳动物', 'source': '动物'}
查询: '微观粒子运动规律' 的结果:

结果 1:
内容: 量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。
元数据: {'source': '物理', 'type': '科学'}


### 4）余弦相似度检索(similarity_search_with_relevance_scores)

- 分数值越接近1（上限），检索到的文档越和问题相似

In [40]:
docs = vector_db.similarity_search_with_relevance_scores("请介绍下一个世界文化遗产和人类建筑奇迹")  # 默认k=4,只选前4个最相关的文本,如要改变,则需要传参赋值

for doc,score in docs:
    print(f"相似度得分:{score:.5f},{doc.page_content},{doc.metadata}")

print("=" * 20)
docs = vector_db.similarity_search_with_relevance_scores("请介绍下一个长城")
for doc,score in docs:
    print(f"相似度得分:{score:.5f},{doc.page_content},{doc.metadata}")

相似度得分:0.73503,长城是中国古代的军事防御工程，总长度超过2万公里。它始建于春秋战国时期，秦朝连接各段，明朝大规模重修。长城是世界文化遗产和人类建筑奇迹。,{'type': '建筑', 'source': '历史'}
相似度得分:0.70473,人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。,{'type': '灵长类', 'source': '生物'}
相似度得分:0.69439,《红楼梦》是中国古典文学四大名著之一，作者曹雪芹。小说以贾、史、王、薛四大家族的兴衰为背景，描绘了贾宝玉与林黛玉的爱情悲剧，反映了封建社会的种种矛盾。,{'source': '文学', 'type': '小说'}
相似度得分:0.65506,狗是人类最早驯化的动物之一，属于犬科。它们具有高度社会性，能理解人类情绪，常被用作宠物、导盲犬或警犬。不同品种的狗在体型、毛色和性格上有很大差异。,{'type': '哺乳动物', 'source': '动物'}
相似度得分:0.78412,长城是中国古代的军事防御工程，总长度超过2万公里。它始建于春秋战国时期，秦朝连接各段，明朝大规模重修。长城是世界文化遗产和人类建筑奇迹。,{'source': '历史', 'type': '建筑'}
相似度得分:0.68500,《红楼梦》是中国古典文学四大名著之一，作者曹雪芹。小说以贾、史、王、薛四大家族的兴衰为背景，描绘了贾宝玉与林黛玉的爱情悲剧，反映了封建社会的种种矛盾。,{'type': '小说', 'source': '文学'}
相似度得分:0.64412,人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。,{'type': '灵长类', 'source': '生物'}
相似度得分:0.62933,量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。,{'source': '物理', 'type': '科学'}


### 5）欧式距离/L2距离分数搜索(similarity_search_with_score)

- 欧式距离(L2距离):衡量两点之间直线距离的方法,也称“欧几里得距离”
- 分数值越小，检索到的文档越和问题相似。分值取值范围：[0，正无穷]

In [49]:
docs = vector_db.similarity_search_with_score(query="请介绍下一个长城")

for doc, score in docs:
    print(f" [L2距离得分={score:.3f}] {doc.page_content} [{doc.metadata}]")

 [L2距离得分=0.305] 长城是中国古代的军事防御工程，总长度超过2万公里。它始建于春秋战国时期，秦朝连接各段，明朝大规模重修。长城是世界文化遗产和人类建筑奇迹。 [{'source': '历史', 'type': '建筑'}]
 [L2距离得分=0.445] 《红楼梦》是中国古典文学四大名著之一，作者曹雪芹。小说以贾、史、王、薛四大家族的兴衰为背景，描绘了贾宝玉与林黛玉的爱情悲剧，反映了封建社会的种种矛盾。 [{'source': '文学', 'type': '小说'}]
 [L2距离得分=0.503] 人类是地球上最具智慧的生物，属于灵长目人科。现代人类（智人）拥有高度发达的大脑，创造了语言、工具和文明。人类的平均寿命约70-80年，分布在全球各地。 [{'source': '生物', 'type': '灵长类'}]
 [L2距离得分=0.524] 量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。 [{'source': '物理', 'type': '科学'}]


### 6）最大边际相关性(max_marginal_relevance_search)

- max_marginal_relevance_search: 即MMR,平衡 相关性 和 多样性 的检索策略，避免返回高度相似的冗余结果

In [48]:
docs = vector_db.max_marginal_relevance_search(
    query="请介绍下红楼梦这本小说",
    # 默认k=4,只选取最符合要求的前4个结果
    # lambda_mult=0.1 # 追求检索结果的最大多样性
    lambda_mult=0.9  # 追求最大相似性
)
#
for i, doc in enumerate(docs, 1):
    print(f"\n📖 结果 {i}:")
    print(f"📌 内容: {doc.page_content}")
    print(f"🏷 标签: {', '.join(f'{k}={v}' for k, v in doc.metadata.items())}")


📖 结果 1:
📌 内容: 《红楼梦》是中国古典文学四大名著之一，作者曹雪芹。小说以贾、史、王、薛四大家族的兴衰为背景，描绘了贾宝玉与林黛玉的爱情悲剧，反映了封建社会的种种矛盾。
🏷 标签: source=文学, type=小说

📖 结果 2:
📌 内容: 长城是中国古代的军事防御工程，总长度超过2万公里。它始建于春秋战国时期，秦朝连接各段，明朝大规模重修。长城是世界文化遗产和人类建筑奇迹。
🏷 标签: type=建筑, source=历史

📖 结果 3:
📌 内容: 葡萄是一种常见的水果，属于葡萄科葡萄属植物。它的果实呈圆形或椭圆形，颜色有绿色、紫色、红色等多种。葡萄富含维生素C和抗氧化物质，可以直接食用或酿造成葡萄。
🏷 标签: type=植物, source=水果

📖 结果 4:
📌 内容: 量子力学是研究微观粒子运动规律的物理学分支。它提出了波粒二象性、测不准原理等概念，彻底改变了人类对物质世界的认知。量子计算机正是基于这一理论发展而来。
🏷 标签: type=科学, source=物理


参数说明:lambda_mult 参数值介于 0 到 1 之间，用于确定结果之间的多样性程度，其中 0 对应最大多样性，1 对应最大相似性。默认值为 0.5。